### Candace Edwards
### ICS 635: Homework 5
### Part 1: [Notebook Link](https://colab.research.google.com/drive/1gz9EQGyhTAOqW9yYsHNSQSdA6yG7kMEE?usp=sharing)
### Part 2: Q4: [Notebook Link](https://colab.research.google.com/drive/1AAUvfyM4YqTZeiikR8t99Se0Wn97afD6?usp=sharing)

In [ ]:
! pip install tensorflow keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import cv2

faces = datasets.fetch_olivetti_faces()

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, utils

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
###
### Load the dataset.
###

X_grey = faces['images']
y = faces['target']

X = []
for a in X_grey:
  image_rgb = cv2.cvtColor(a, cv2.COLOR_GRAY2RGB) 
  X.append(image_rgb)
X = np.array(X)

y = utils.to_categorical(
    y, num_classes=40, dtype='float32'
)


###
### Split data into train and test sets. Do not change. For reference only.
###

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [ ]:
###
### Sourced from class notes
###
def plot_acc(history, ax = None, xlabel = 'Epoch #'):
    if hasattr(history, 'history_'):
      history = history.history_
    else:
      history = history.history
    history.update({'epoch':list(range(len(history['val_accuracy'])))})
    history = pd.DataFrame.from_dict(history)

    best_epoch = history.sort_values(by = 'val_accuracy', ascending = False).iloc[0]['epoch']

    if not ax:
      f, ax = plt.subplots(1,1)
    sns.lineplot(x = 'epoch', y = 'val_accuracy', data = history, label = 'Validation', ax = ax)
    sns.lineplot(x = 'epoch', y = 'accuracy', data = history, label = 'Training', ax = ax)
    ax.axhline(0.5, linestyle = '--',color='red', label = 'Chance')
    ax.axvline(x = best_epoch, linestyle = '--', color = 'green', label = 'Best Epoch')  
    ax.legend(loc = 7)    
    ax.set_ylim([0.4, 1])

    ax.set_xlabel(xlabel)
    ax.set_ylabel('Accuracy (Fraction)')
    
    plt.show()

In [ ]:
###
### CNN Implementation
### 

def get_neural_network(X_train, y_train):
  """
  Define, train, and return the neural network.

  You may replace this code with any TensorFlow model that you wish.
  """

  model = models.Sequential()
  model.add(layers.Conv2D(65, (3, 3), activation='relu', input_shape=(64, 64, 3)))
  ##model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(64, 64, 3)))

  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.05)) #added dropout layer [0.05,0.15,0.15,0.05,0.07]
  '''
  model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
  '''
  #changed to paddin =same
  model.add(layers.Conv2D(64,(3,3), activation ='relu', padding="same"))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.005)) #[0.005, 0.15,0.20,0.005,0.05]

  #model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Conv2D(32, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Dropout(0.005)) #[0.0005, 0.15,0.30]
  ##
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(40, activation='softmax'))

  model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
  
  model.fit(X_train, y_train, 
          epochs=34)
  

  

  # #validation data: resplit traininging data
  # X_train, X_val,y_train,y_val = train_test_split(X_train,y_train,test_size= 0.25,random_state=10)

  # history = model.fit(X_train, y_train, \
  #                   validation_data=(X_val, y_val), \
  #                   epochs=40)
  # plot_acc(history)
  # ##

  return model

In [ ]:
###
### Train the neural network.
###

model = get_neural_network(X_train, y_train)

Epoch 1/34
10/10 [==============================] - 5s 379ms/step - loss: 3.7020 - accuracy: 0.0367
Epoch 2/34
10/10 [==============================] - 5s 538ms/step - loss: 3.6887 - accuracy: 0.0300
Epoch 3/34
10/10 [==============================] - 4s 376ms/step - loss: 3.6872 - accuracy: 0.0233
Epoch 4/34
10/10 [==============================] - 4s 366ms/step - loss: 3.6812 - accuracy: 0.0467
Epoch 5/34
10/10 [==============================] - 5s 530ms/step - loss: 3.6758 - accuracy: 0.0267
Epoch 6/34
10/10 [==============================] - 4s 363ms/step - loss: 3.6707 - accuracy: 0.0367
Epoch 7/34
10/10 [==============================] - 4s 371ms/step - loss: 3.6608 - accuracy: 0.0733
Epoch 8/34
10/10 [==============================] - 5s 533ms/step - loss: 3.6502 - accuracy: 0.0333
Epoch 9/34
10/10 [==============================] - 4s 363ms/step - loss: 3.6167 - accuracy: 0.0567
Epoch 10/34
10/10 [==============================] - 4s 369ms/step - loss: 3.5504 - accuracy: 0.0533

In [ ]:
###
### Get the result on the test set.
###

y_pred = model.predict(X_test).argmax(1)
y_true = y_test.argmax(1)

print('Predicted values: ', y_pred)
print('Actual values: ', y_true)

print('F1 score: ', f1_score(y_true, y_pred, average='weighted'))

4/4 [==============================] - 1s 123ms/step
Predicted values:  [20 28  3 21  9  8 32  9 26 12  0 36  5  7 13  4 27 37 23 38  7  1 39 25
  0 20 11 22 26 14 39  3 26  5 23 11  8 34 15 14  9  5  7 36  8 38 14 18
  2 17  4 32 33  7 37  3 22 22  3 15 12 29 36 20 10  3 35 26 39  7 32 14
  0  4 38 24 22 36 17 28 12  1 20 36 27  6 24 30 10  9 23 33 11 22 18 31
 37 38 23 15]
Actual values:  [20 28  3 21  9  8 32  9 26 12  0 36  5  7 13  4 27 37 23 38  7  1 39 27
  0 39 11 22 26 10 39 19 26  5 23 11 11 34 15 14 38  5  7  2  8 38 14 18
  2 17  4 32 33  7 37  3 22 17  3 15 12 29 25  7 10  3 35 26 39  7 32 14
  0  4 38 24 22 36 17 28  0  1 20 25 27  6 24 30 10  9 23 33 11 22 18 31
 37 38 23  7]
F1 score:  0.8662698412698413


In [ ]:
###
### Save model and weights. These will be used for grading.
### The weights will be downloaded to your computer. Use Chrome.
###

# TODO: CHANGE TO YOUR NAME 
student_name = 'candace_edwards'

model.save(student_name + '_model.h5')

from google.colab import files
files.download(student_name + '_model.h5') 